# Capstone Project- Fake News Identification

### Web scraping  news articles
Here, I am using Webhoseio API to scrape news articles available on https://mediabiasfactcheck.com. Here, I am scraping news articles from different sites having factual reporting type High, Mixed and Low. The ***mediabiasfactcheck.com*** does this work manually.

In [224]:
import pandas as pd
import numpy as np

import requests



Below code is scraping news articles from diffrent websites. Here the websites are given as string in query parameter.
This code is used to scrape the news articles from past 30 days(from the day of crawled).

In [252]:
import webhoseio

#define API key
webhoseio.config(token="d4feac70-4b0a-4896-948c-96885d5a962e")

#defining query parameters.
#q: list of sites to scrape
#ts: time stamp of last 30 days(defined by webhoseio API)
#sort: it returns the news articles sorted by the order they are crawled
query_params = {
    "q": "site:ageofautism.com OR site:10news.one OR site:100percentfedup.com OR site:allnewspipeline.com OR site:americanthinker.com OR site:bannedinformation.com OR site:breaking911.com OR site:eaglerising.com OR site:southafricatoday.net OR site:subjectpolitics.com OR site:ecowatch.com OR site:ancient-code.com OR site:cnn.com OR site:thenewcivilrightsmovement.com OR site:splinternews.com OR site:buzzfeed.com site_category:news OR site:politicaldig.com OR site:ilovemyfreedom.org OR site:trueviralnews.com OR site:altright.com OR site:censored.news OR site:canadafreepress.com OR site:syrianews.cc OR site:bustedlocals.com OR site:bostonreview.net OR site:rabble.ca OR site:healthline.com OR site:abc.net.au OR site: japantimes.co.jp OR site:cnbc.com OR site:livescience.com",
    "ts": "1554956406114",
    "sort": "crawled"
    }

# return the JSON of all scraped news article
output = webhoseio.query("filterWebContent", query_params)


####  Preparing the data return by JSON object to to put it in CSV and dataframe for later use
Here the return Json object can onlu hold 100 news article(posts) at a time so to get all the data we need to call webhoseio.get_next() method and iterate it untill all news articles are processed

In [259]:
final_list = []
while len(output['posts'])>0:
    for index,each_post in enumerate(output['posts']):
        final_dict = {}
        final_dict['url'] = output['posts'][index]['thread']['url']
        final_dict['site'] = output['posts'][index]['thread']['site_full']
        final_dict['site_categories'] = output['posts'][index]['thread']['site_categories']
        final_dict['title'] = output['posts'][index]['thread']['title']
        final_dict['published'] = output['posts'][index]['thread']['published']
        final_dict['participants_count'] = output['posts'][index]['thread']['participants_count']
        final_dict['site_type'] = output['posts'][index]['thread']['site_type']
        final_dict['country'] = output['posts'][index]['thread']['country']
        final_dict['main_image'] = output['posts'][index]['thread']['main_image']
        final_dict['domain_rank'] = output['posts'][index]['thread']['domain_rank']
        final_dict['facebook_likes'] = output['posts'][index]['thread']['social']['facebook']['likes']
        final_dict['facebook_comments'] = output['posts'][index]['thread']['social']['facebook']['comments']
        final_dict['facebook_comments'] = output['posts'][index]['thread']['social']['facebook']['shares']
        final_dict['author'] = output['posts'][index]['author']
        final_dict['title'] = output['posts'][index]['title']
        final_dict['text'] = output['posts'][index]['text']
        final_dict['language'] = output['posts'][index]['language']
        final_dict['rating'] = output['posts'][index]['rating']
        final_dict['rating'] = output['posts'][index]['rating']
        final_dict['crawled'] = output['posts'][index]['crawled']
        final_list.append(final_dict)
    output = webhoseio.get_next()
        

In [260]:
#Total scraped articles
len(final_list)

34680

In [261]:
df_final = pd.DataFrame(final_list)

In [263]:
df_final.shape

(34680, 17)

#### Label the articles based on their factual report
After scraping articles we need to label them as the JSON object donot return type of factual reporting each article. Therefore, I am labeling each articles based on which site they belong(factual report of each of this site is given in https://mediabiasfactcheck.com.


In [267]:
#create lists of websites based on their factual report
high_factual_reporting_sites = ['cnbc','abc','japantimes','livescience','rabble','bostonreview','healthline']
mixed_factual_reportng_sites = ['ecowatch','buzzfeed','cnn','splinternews','ancient-code','politicaldig.com','thenewcivilrightsmovement']
low_factual_reportng_sites = ['trueviralnews','ilovemyfreedom','syrianews','americanthinker','southafricatoday','breaking911','100percentfedup','ageofautism','allnewspipeline']

In [275]:
#define a function to lable each article
def factual_report (row):
    for each_value in high_factual_reporting_sites:
        if each_value in row['site']:
            return 'High'
    for each_value in mixed_factual_reportng_sites:
        if each_value in row['site']:
            return 'Mixed'
    for each_value in low_factual_reportng_sites:
        if each_value in row['site']:
            return 'Low'


In [270]:
df_final['factual_reporting'] = df_final.apply (lambda row: factual_report(row), axis=1)


In [271]:
## labled articles in each category
df_final['factual_reporting'].value_counts()

Mixed    17159
High     13035
Low       4486
Name: factual_reporting, dtype: int64

In [273]:
# moving this information to csv file for further use
df_final.to_csv('./data/news_data.csv')